In [ ]:
pip install faiss-cpu


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 72.7 MB/s eta 0:00:00


In [ ]:
import os
import json
import re
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer

# Files to include
files = {
    "english_ada": "ada_diabetes_guidelines.txt",
    "english_cdc": "cdc_diabetes_guidelines.txt",
    "english_who": "who_diabetes_guidelines.txt",
    "english_risk_factors": "diabetes_risk_factors.txt",
    "spanish_ndep": "spanish_guidelines_ndep.txt",
    "spanish_who": "spanish_guidelines_who",
    "spanish_cdc": "spanish_guidelines_cdc.txt"
}

# Chunking function
def chunk_text(text, max_words=150, overlap=30):
    paragraphs = re.split(r'\n{2,}', text)
    chunks = []
    current_chunk = []

    for para in paragraphs:
        words = para.split()
        if not words:
            continue

        while words:
            space_left = max_words - len(current_chunk)
            chunk_part = words[:space_left]
            current_chunk.extend(chunk_part)
            words = words[space_left:]

            if len(current_chunk) >= max_words:
                chunks.append(" ".join(current_chunk))
                current_chunk = current_chunk[-overlap:]

    if current_chunk:
        chunks.append(" ".join(current_chunk))

    return chunks

# Prepare separate lists
english_chunks, spanish_chunks = [], []
english_texts, spanish_texts = [], []

# Process each file
for name, path in files.items():
    if os.path.exists(path):
        print(f"Reading and chunking: {name}")
        with open(path, "r", encoding="utf-8") as f:
            text = f.read()
        chunks = chunk_text(text)
        lang = "es" if "spanish" in name else "en"
        for chunk in chunks:
            chunk_entry = {"text": chunk, "lang": lang, "source": name}
            if lang == "es":
                spanish_chunks.append(chunk_entry)
                spanish_texts.append(chunk)
            else:
                english_chunks.append(chunk_entry)
                english_texts.append(chunk)
    else:
        print(f"File not found: {path}")

print(f"English chunks: {len(english_chunks)}")
print(f"Spanish chunks: {len(spanish_chunks)}")

# Load multilingual model
print("Embedding with multilingual MiniLM...")
model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

# Encode and build FAISS index for English
print("Embedding English...")
en_embeds = model.encode(english_texts, show_progress_bar=True, convert_to_numpy=True)
en_index = faiss.IndexFlatL2(en_embeds.shape[1])
en_index.add(np.array(en_embeds, dtype="float32"))

faiss.write_index(en_index, "rag_en.index")
with open("rag_passages_en.json", "w", encoding="utf-8") as f:
    json.dump(english_chunks, f, indent=2, ensure_ascii=False)
print("English FAISS and passages saved.")

# Encode and build FAISS index for Spanish
print("Embedding Spanish...")
es_embeds = model.encode(spanish_texts, show_progress_bar=True, convert_to_numpy=True)
es_index = faiss.IndexFlatL2(es_embeds.shape[1])
es_index.add(np.array(es_embeds, dtype="float32"))

faiss.write_index(es_index, "rag_es.index")
with open("rag_passages_es.json", "w", encoding="utf-8") as f:
    json.dump(spanish_chunks, f, indent=2, ensure_ascii=False)
print("Spanish FAISS and passages saved.")

Reading and chunking: english_ada
Reading and chunking: english_cdc
Reading and chunking: english_who
Reading and chunking: english_risk_factors
Reading and chunking: spanish_ndep
File not found: spanish_guidelines_who
Reading and chunking: spanish_cdc
English chunks: 2705
Spanish chunks: 212
Embedding with multilingual MiniLM...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.89k [00:00<?, ?B/s]

...

Embedding English...


Batches:   0%|          | 0/85 [00:00<?, ?it/s]

English FAISS and passages saved.
Embedding Spanish...


Batches:   0%|          | 0/7 [00:00<?, ?it/s]

Spanish FAISS and passages saved.


In [ ]:
!pip install faiss-cpu